In [1]:
import constants.constants as constants
import prompts.prompt_classifier as prompt_classifier
import prompts.prompt_banking as prompt_banking
import prompts.prompt_cards as prompt_card
import prompts.prompt_oeffnungszeiten as prompt_oeffnungszeiten
import functions.functions as functions

from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.tools import tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType

from langchain.prompts import ChatPromptTemplate
from langchain.utilities import SQLDatabase
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [2]:
chat = ChatOpenAI(openai_api_key=constants.OPENAI_API_KEY)


@tool
def debit_card_information(question):
    """Beantwortet Fragen zu dem Thema: Sperren von Bankomat-, Debit- und Kreditkarten."""
    messages = [
        SystemMessage(content=prompt_card.prompt_cards),
        HumanMessage(content=question)
    ]

    return chat.predict_messages(messages)
@tool
def banking_information(question):
    """Beantwortet Fragen zu dem Thema: Sperren des Online-Bankings."""
    messages = [
        SystemMessage(content=prompt_banking.prompt_banking),
        HumanMessage(content=question)
    ]

    return chat.predict_messages(messages)



In [3]:
db = SQLDatabase.from_uri("sqlite:///./database/raika.db")

In [4]:
def get_schema(_):
    return db.get_table_info()

In [5]:
def run_query(query):
    return db.run(query)

In [6]:
## wir arbeiten hier
@tool
def banking_oeffnungszeiten(question):
    """Beantwortet Fragen zu dem Thema: Öffnungszeiten von Banken."""
    template = """Based on the table schema below, write a SQL query that would answer the user's question:
    {schema}

    Question: {question}
    SQL Query:"""
    prompt = ChatPromptTemplate.from_template(template)
    model = ChatOpenAI(openai_api_key=constants.OPENAI_API_KEY)

    sql_response = (
        RunnablePassthrough.assign(schema=get_schema)
        | prompt
        | model.bind(stop=["\nSQLResult:"])
        | StrOutputParser()
    )
    sql_response.invoke({"question": question})
    template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
    {schema}

    Question: {question}
    SQL Query: {query}
    SQL Response: {response}"""
    prompt_response = ChatPromptTemplate.from_template(template)
    full_chain = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | model
    )
    
    return full_chain.invoke({"question": question})


In [8]:
# import time
tools = [debit_card_information, banking_information, banking_oeffnungszeiten]
messages = [
     SystemMessage(content=prompt_classifier.system_message),
    first_ai_msg := AIMessage(content="Hier ist der Chatbot der Raiffeisenlandesbank Niederösterreich-Wien. Gerne beantworte ich Ihre Fragen zur Sperre von Debit- und Kreditkarten sowie zur Sperre des Online-Bankings."),
 ]


# print(first_ai_msg.content)
# question_of_user = str(input(""))
# start = time.time()
# messages.append(HumanMessage(content=question_of_user))


# mrkl = initialize_agent(tools, chat, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)
# print(mrkl.run(messages))
# end = time.time()

# print(end - start)